# Installing DEAP


Assuming that your other package versions are compatible with DEAP:

1. Clone the repo https://github.com/DEAP/deap.git

2. Open a command window in the folder where the DEAP setup.py file is located and type the following: 

        conda install -c conda-forge pypandoc
        python setup.py install



In [2]:
import matplotlib.pyplot as plt
import sys
import array
import random
import numpy as np

from deap import  algorithms
from deap import  base
from deap import  creator
from deap import  tools

%matplotlib inline

## Logbook to Pandas DataFrame

Logbook to pandas for multistats with chapters.  

https://stackoverflow.com/questions/42904904/saving-deap-results-into-pandas-dataframe


In [3]:
import pandas as pd
from functools import reduce
from operator import add, itemgetter

def logtoDF(log):
    df = None
    chapter_keys = log.chapters.keys()
    if chapter_keys:
        sub_chaper_keys = [c[0].keys() for c in log.chapters.values()]


        data = [list(map(itemgetter(*skey), chapter)) for skey, chapter in zip(sub_chaper_keys, log.chapters.values())]
        data = np.array([[*a, *b] for a, b in zip(*data)])

        columns = reduce(add, [["_".join([x, y]) for y in s] for x, s in zip(chapter_keys, sub_chaper_keys)])
        df = pd.DataFrame(data, columns=columns)

        keys = log[0].keys()
        data = [[d[k] for d in logbook] for k in keys]
        for d, k in zip(data, keys):
            df[k] = d
    else:
        data = [[i for i in item.values()] for item in log]
        df = pd.DataFrame(data, columns=log.header)
    
    return df


        

## Attaching names to the cities

https://stackoverflow.com/questions/47429746/how-to-create-list-of-city-as-individual-in-deap-python

In [4]:
def generate_individual():
    return ["Almeria","Cadiz","Cordoba","Granada","Huelva","Jaen","Malaga", "Seville"]

creator.create("FitnessMin", base.Fitness,weights=(-1.0,))
creator.create("Individual",list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("indices", generate_individual) # Gen, in this case, a number which represents a city.
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices) # Define a route of cities. A chromosome.
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
POP_SIZE = 50
pop = toolbox.population(n=POP_SIZE)
print(pop)

[['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['A